In [1]:
import LSTM_train
import UpdateStockPrices

import datetime
import numpy as np
import pandas as pd
from io import StringIO

import os
import joblib

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

import psycopg2

In [2]:
def Create_tensor(data):
    tensor=np.asarray(data)
    tensor=tensor.reshape(1,tensor.shape[0],1)
    return np.asarray(tensor,dtype=object).astype(np.float)

In [3]:
def Connect():
    connection = psycopg2.connect(user="postgres",
        password="adminpw",
        host="127.0.0.1",
        port="5432",
        database="Thesis")
    curr = connection.cursor()
    return connection, curr

In [4]:
UpdateStockPrices.Update_databases()

[*********************100%***********************]  7 of 7 completed


'No new data is available to update table'

In [5]:
connection, curr =Connect()
sql='SELECT * FROM stockprices'
stocks=pd.read_sql_query(sql,connection)
connection.close()
curr.close()

In [7]:
stock_date=stocks.iloc[-1,0]  
with open('../../Models/trained.txt','r') as f:
    last_date=f.read()
    if last_date!=stock_date.strftime('%d-%m-%Y'):
        NEEDS_UPDATE=True
    else:
        NEEDS_UPDATE=False

In [ ]:
if NEEDS_UPDATE==True:
    for col in stocks.columns:
        if col=='date':
            continue
        LSTM_train.Start_training(col)
        
    with open('../../Models/trained.txt','w') as f:
        trained_date=stocks.iloc[-1,0]   
        f.write(trained_date.strftime('%d-%m-%Y'))

In [10]:
connection, curr =Connect()
curr.execute('SELECT max(last_observed_date) FROM predictions')
maxdate=np.squeeze(curr.fetchall())
if maxdate==stock_date:
    PREDICTION_NEEDED=False
else:   
    PREDICTION_NEEDED=True
connection.close()
curr.close()

In [11]:
sql=f'SELECT * FROM stockprices order by date desc limit 60'

In [12]:
connection, curr =Connect()
df = pd.read_sql_query(sql,connection)
connection.close()
curr.close()

In [19]:
dates=pd.date_range(stock_date+datetime.timedelta(days=1),stock_date+datetime.timedelta(days=9))

In [13]:
cols=[columns for columns in stocks.columns if columns!='date']
preds_dict=dict((col, [0]) for col in cols)

In [ ]:
Scaler=MinMaxScaler(feature_range = (-1, 1))
tf.get_logger().setLevel(0)
for symbol in stocks.columns:
    if symbol!='date':
        with open(f'..\..\Data\Weights\{symbol}_opt_weight.txt','r') as f:
            lines=(line.rstrip() for line in f) 
            opt_weights=list(line for line in lines if line)
            opt_weights=np.array(opt_weights).astype(np.float)
            

        #Scaler = joblib.load(f'../../Data/Scaler/{symbol}_scaler.save') 
           
        default_model=tf.keras.models.load_model(f'..\..\Models\Default LSTM\{symbol}') 
        base_model=tf.keras.models.load_model(f'..\..\Models\Base LSTM\{symbol}') 
        gelu_model=tf.keras.models.load_model(f'..\..\Models\Gelu LSTM\{symbol}') 
        swish_model=tf.keras.models.load_model(f'..\..\Models\Swish LSTM\{symbol}') 
            
        hist_list=list(df[symbol])   
        Scaler=Scaler.fit(np.array(hist_list).reshape(-1,1))  
        hist_list=list(np.array(hist_list).reshape(-1,1))    
        for i in range(9):             
            hist=Create_tensor(hist_list[-60:])
            preds=[default_model.predict(hist),base_model.predict(hist),gelu_model.predict(hist),swish_model.predict(hist)]
            pred_scaled=opt_weights.dot(np.squeeze(preds)).reshape(-1,1)
            preds_dict[symbol].append(np.squeeze(Scaler.inverse_transform(pred_scaled)[0]))
            hist_list.append(pred_scaled)
            
        final_preds=Scaler.inverse_transform(np.array(hist_list).reshape(-1,1))

In [20]:
result=pd.DataFrame.from_dict(preds_dict)[1:]

In [21]:
result['last_observed_date']=stock_date
result['date']=dates.strftime('%Y-%m-%d')

In [22]:
result

,spy,gme,jpm,t,msft,goog,aapl,last_observed_date,date
1,414.22463585347134,246.22255703074555,163.19365701008041,35.009929185344255,245.597501120185,2347.9344600578033,125.33275667448582,2021-05-20,2021-05-21
2,414.86778983091153,285.14460742287383,161.9736652130453,35.0891972929884,254.46679605503118,2250.3906877389322,125.93047097230465,2021-05-20,2021-05-22
3,415.8337437163599,227.8537850927552,164.350316242404,35.202885346440674,263.6277491690506,2242.7035013911905,134.2730032686834,2021-05-20,2021-05-23
4,416.0096032989988,260.82521132208626,164.05133344119415,35.25118724290369,250.46466400752922,2274.2513341884,132.0084754951178,2021-05-20,2021-05-24
5,417.17763867687586,231.27301690678865,163.1428793144608,35.28366750977193,250.65050928000394,2486.060124557408,128.94257645318496,2021-05-20,2021-05-25
6,417.52461851735023,245.85679354795178,165.45002833525277,35.30141188754623,250.69405820468123,2353.9072959216564,136.4917290606769,2021-05-20,2021-05-26
7,418.46908615264914,273.40068470541866,164.14752441039013,35.28257644464265,259.69765173380796,2362.6449586889426,125.32516122019963,2021-05-20,2021-05-27
8,418.5218644822058,200.0180830118471,163.2176348458726,35.248743274750744,254.39047668338497,2362.8401835758773,125.63577829726016,2021-05-20,2021-05-28
9,421.21673653292504,313.6917317953006,162.00949999187182,35.20669710406408,250.7375658954055,2270.1756783984897,125.92390249550924,2021-05-20,2021-05-29


In [23]:
sio = StringIO()
sio.write(result.to_csv(index=None, header=None))
sio.seek(0)
connection, curr =Connect()   
with curr as c:
    c.copy_from(sio, "predictions", columns=result.columns, sep=',')
    connection.commit()
    

In [24]:
for symbol in stocks.columns:
    if symbol!='date': 
        connection, curr =Connect()  
        curr.execute(f'DELETE FROM {symbol}')
        
        sym_df=pd.read_json(r'..\..\Data\Models\\' + symbol + '.json')
        sio = StringIO()
        sio.write(sym_df.to_csv(index=None, header=None))
        sio.seek(0)
        with curr as c:
            c.copy_from(sio, symbol, columns=sym_df.columns, sep=',')
            connection.commit()   
             
connection.close()
curr.close()